In [1]:
%load_ext autoreload
%autoreload 2

import librosa
import pandas as pd
import numpy as np
import os
import h5py
import tensorflow as tf
from tensorflow import keras
import tensorflow_ranking as tfr

from os import path
from config import *


2024-10-02 11:36:29.272348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 11:36:29.272389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 11:36:29.418753: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 11:36:29.746947: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-02 11:36:31.500732: W tensorflow/comp

In [2]:
def get_frames(hdf5_dataset):
    recordings = np.array(hdf5_dataset)
    rec =  recordings[0]
    embeddings_frames = []
    for rec in recordings:
        embeddings_frames.extend([
            (rec, i) for i in range(hdf5_dataset[rec]['X'].shape[0])
        ])
        
    return embeddings_frames
    
    
hdf5_dataset = h5py.File(INTERMEDIATE / 'embeddings_20p.hdf5', 'r')
# embeddings_frames = get_frames(hdf5_dataset)
len(hdf5_dataset)

354

### Labelled, unlabelled and test set init

In [13]:
np.random.seed(0)
np.random.shuffle(embeddings_frames) 

X = np.array([hdf5_dataset[rec]['X'][frame, :] for (rec, frame) in embeddings_frames])
Y = np.array([hdf5_dataset[rec]['Y_strict'][:, frame] for (rec, frame) in embeddings_frames])
n_frames = len(X) 

In [14]:
# verify
X[0].shape, Y[0].shape

((1024,), (4,))

In [19]:
n_frames = len(X) 
test_cut =  int(n_frames * 0.8)
label_cut = int(test_cut * 0.8)

# test data
train_X, train_Y = X[:test_cut],  Y[:test_cut]
test_X, test_Y = X[test_cut:],  Y[test_cut:]

# labelled and unlabelled data
unlabelled_X, unlabelled_Y = train_X[:label_cut], train_Y[:label_cut]
labelled_X, labelled_Y = train_X[label_cut:], train_Y[label_cut:]

u, t = len(unlabelled_X), len(labelled_Y),  
print(f'{u} unlabelled \n {t} labelled \n {(t/u)*100:.2f}% initial labelling budget')
print(f'\ntest XY lens: {len(test_X), len(test_Y)}')
print(f'unlabelled XY lens: {len(unlabelled_X), len(unlabelled_Y)}')
print(f'labelled XY lens: {len(labelled_X), len(labelled_Y)}')

282931 unlabelled 
 70733 labelled 
 25.00% initial labelling budget

test XY lens: (88416, 88416)
unlabelled XY lens: (282931, 282931)
labelled XY lens: (70733, 70733)


### Model & Training Utils

In [16]:
from keras.layers import Input, Dense, BatchNormalization
from functools import partial
from keras import metrics
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from util import DEFAULT_TOKENS

def create_model() -> keras.Model:
    default_dense =  partial(Dense, activation='relu')
                            #  kernel_initializer=keras.initializers.LecunNormal(seed=0)) 
    
    return keras.Sequential([
            Input(shape=(1024,)),
            default_dense(512), 
            default_dense(256), 
            default_dense(64), 
            Dense(4, activation='sigmoid',)
    ])

def compile(model):
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',  
        metrics=[
            metrics.Recall(thresholds=0.5),
            metrics.Precision(thresholds=0.5),
            metrics.AUC(curve='pr', name='auc_pr'),
            tfr.keras.metrics.get(key="map", name="metric/map"),
        ]
    )

def oversample_minority_classes(X, Y):
    num_classes = Y.shape[1]
    class_counts = np.sum(Y, axis=0)
    max_count = np.max(class_counts)
    new_X = X
    new_Y = Y
    for class_index in range(num_classes): # 0 1 2 3
        class_indices = np.where(Y[:, class_index] == 1)[0] # locations of nr
        num_samples_needed = max_count - len(class_indices)
        if num_samples_needed > 0:
            sampled_indices = np.random.choice(class_indices, num_samples_needed, replace=True)
            sampled_X = X[sampled_indices]
            sampled_Y = Y[sampled_indices]
            
            new_X = np.vstack((new_X, sampled_X))
            new_Y = np.vstack((new_Y, sampled_Y))
            
    return new_X, new_Y


def undersample(X, Y, reduce_to=0.5): 
    total_instances = len(X)
    assert len(Y) == total_instances
    annotated_mask = np.any(Y, axis=1)
    annotated_X = X[annotated_mask]
    annotated_Y = Y[annotated_mask]
    unannotated_mask = ~annotated_mask
    unannotated_X = X[unannotated_mask]
    unannotated_Y = Y[unannotated_mask]
    
    n_unannotated_to_sample = int(reduce_to * len(unannotated_X))
    if len(unannotated_X) > 0: 
        sampled_indexes = np.random.choice(len(unannotated_X), size=n_unannotated_to_sample, replace=False)
        sampled_X = unannotated_X[sampled_indexes]
        sampled_Y = unannotated_Y[sampled_indexes]
        new_X = np.vstack((annotated_X, sampled_X))
        new_Y = np.vstack((annotated_Y, sampled_Y))
    else:
        new_X = annotated_X
        new_Y = annotated_Y
    
    return new_X, new_Y


def train(model, X, Y, model_dir, reduce_empty_class, stopping_patience=5, 
          stopping_moniter='loss', epochs=10, **kwargs):
    model_dir.mkdir(exist_ok=True)

    # tensorboard
    log_dir = model_dir / "logs" / "fit"
    log_dir.mkdir(parents=True, exist_ok=True)
    tensorboard_callback = TensorBoard(log_dir=log_dir)

    # checkpoints
    checkpoint_path = model_dir / "training"
    Path(checkpoint_path).mkdir(exist_ok=True)  
    cp_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_path / 'checkpoint.weights.h5', 
        save_weights_only=True,
        verbose=1, 
    )

    # early stopping
    es_checkpoint = EarlyStopping(
        monitor=stopping_moniter,
        patience=stopping_patience,
        restore_best_weights=True,
    )
    
    histories = []
    for epoch in range(epochs): 
        print(f"--- Epoch {epoch + 1} ---")
        
        # reduce the amount of unlabelled instances by 80%
        print(f'previous XY shapes: ', X.shape, Y.shape)
        resampled_X, resampled_Y = undersample(X, Y, reduce_to=reduce_empty_class)
        resampled_X, resampled_Y = oversample_minority_classes(resampled_X, resampled_Y)
        print(f'resampled XY shapes: ', resampled_X.shape, resampled_Y.shape)
        for i in range(4):
            token_order = list(DEFAULT_TOKENS.keys())
            print(f'total of {token_order[i]}) = {resampled_Y[:, i].sum()}')
        
        history = model.fit(x=resampled_X, y=resampled_Y, 
                  **kwargs, 
                  epochs=1, 
                  verbose=2,
                    callbacks=[tensorboard_callback, cp_callback, es_checkpoint])
        
        histories.append(history)
        
    df = pd.DataFrame(history.history)
    df.to_csv(model_dir / "history.csv")
    model.save(model_dir / 'model.keras')
    return histories


In [ ]:
### test that it works

model = create_model()
compile(model)

history = train( 
    model, train_X, train_Y, # essentially training data 
    epochs=5,
    validation_data=(test_X, test_Y),
    batch_size=32,
    model_dir=MODEL_DIR / 'testing', 
    reduce_empty_class=0.9,
    stopping_moniter='val_loss'
)

pred_Y = model(test_X)

from util import DEFAULT_TOKENS
token_order = list(DEFAULT_TOKENS.keys())
T = 0.5

# look at all precisions and recalls
for i in range(4):
    print(token_order[i])
    m = keras.metrics.Precision(thresholds=T)
    m.update_state(test_Y[:, i], pred_Y[:, i])
    print('Precision = ', m.result().numpy())

    m = keras.metrics.Recall(thresholds=T)
    m.update_state(test_Y[:, i], pred_Y[:, i])
    print('Recall = ', m.result().numpy())
    print("")

In [ ]:
w = 5

print("RECALL")
for i in (0, 3):
    print(f'\n------ {token_order[i]} -------')
    y_true = np.array(test_Y)
    pred_Y = np.array(pred_Y)
    y_pred_rounded = np.round(pred_Y, 3)

    rows = np.where(test_Y[:, i])[0]

    for r in rows[:10]:
        print(f"\nat r={r}")
        print(y_true[r - w: r + w, i].T)
        print( y_pred_rounded[r - w: r + w, i].T)
        
    
print("\nPREC")
for i in (0, 3):
    print(f'\n------ {token_order[i]} -------')
    y_true = np.array(test_Y)
    pred_Y = np.array(pred_Y)
    y_pred_rounded = np.round(pred_Y, 3)

    rows = np.where(pred_Y[:, i] > 0.5)[0]

    for r in rows[:5]:
        print(f"\nat r={r}")
        print(y_true[r - w: r + w, i].T)
        print(y_pred_rounded[r - w: r + w, i].T)

### AL utils

In [75]:
import gc

def AL_simulation(unlabelled: tuple, intial_labelled: tuple, test: tuple,
                    model, name, sampling_query, pool_size, evaluate_metrics,
                    num_iterations=10, epochs=10):
    
    
    
    unlabelled_X, unlabelled_Y = unlabelled
    labelled_X, labelled_Y = intial_labelled
    test_X, test_Y = test
    
    histories = []
    iteration_metrics = {}
    dir = MODEL_DIR / name
    Path(dir).mkdir(exist_ok=True)
    
    for i in range(num_iterations):
        print(f" --- Iteration {i + 1} --- ")
        
        # model is recompiled per epoch
        model = create_model()
        compile(model)
        
        history = train( 
            model, labelled_X, labelled_Y,
            epochs=epochs,
            batch_size=32,
            model_dir=MODEL_DIR / dir, 
        )
        histories.append(history)
        
        # get metrics from test set
        y_pred = model.predict(test_X)
        y_true = test_Y
        budget = (len(labelled_Y) / len(train_X))
        iteration_metrics[f'labelling_budget={budget:.2f}'] = evaluate_metrics(y_true, y_pred)
        
        # active learning query on unlabelled set
        y_pred = model.predict(unlabelled_X)
        indexes_pool = sampling_query(y_pred, unlabelled_X, pool_size)
        X_new = [unlabelled_X[i] for i in indexes_pool]
        Y_new = [unlabelled_Y[i] for i in indexes_pool]
        
        # # add the newly 'annotated' samples to the labelled set        
        labelled_X = np.vstack((labelled_X, X_new))
        labelled_Y = np.vstack((labelled_Y, Y_new))
        
        # remove the new samples from the unlabelled set
        mask = np.ones(len(unlabelled_X), dtype=bool)
        mask[indexes_pool] = False
        unlabelled_X = unlabelled_X[mask]
        unlabelled_Y = unlabelled_Y[mask]
        
        print("current iteration metrics: ", iteration_metrics)
        pd.DataFrame(history.history).to_csv(dir / 'hist.csv')
        np.save(dir / 'metrics.npy', iteration_metrics)
        gc.collect()

    return model, iteration_metrics, histories 

### Xgboost test

In [59]:
import numpy as np
import xgboost as xgb
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

train_X, train_Y = unlabelled_X, unlabelled_Y
resampled_X, resampled_Y = undersample(X, Y, reduce_to=0.5) # NOTE try reducing this?
resampled_X, resampled_Y = oversample_minority_classes(resampled_X, resampled_Y)

# Train a separate XGBoost model for each class
models = []
for i in range(Y.shape[1]):
    print(i)
    
    scale_pos_weight = (resampled_Y.shape[0] / resampled_Y[:, i].sum())
    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbose=1,
                              scale_pos_weight=scale_pos_weight)
    
    model.fit(train_X, train_Y[:, i])
    models.append(model)


# Make predictions
pred_Y = np.array([model.predict(test_X) for model in models]).T

# Evaluate the model
f1 = f1_score(test_Y, pred_Y, average='macro')
accuracy = accuracy_score(test_Y, pred_Y)

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

0


/home/ec2-user/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [07:11:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


1


/home/ec2-user/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [07:12:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


2


/home/ec2-user/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [07:12:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


3


/home/ec2-user/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [07:13:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


F1 Score: 0.3081
Accuracy: 0.9860


In [73]:
from sklearn.metrics import precision_score, recall_score

pred_Y = np.array([model.predict(test_X) for model in models]).T
print(pred_Y.shape)

# Calculate precision, recall, and AUC for PR curve for each class
for i in range(Y.shape[1]):
    
    y_pred = (pred_Y[:, i] > 0.5).astype(int)

    # Calculate precision
    precision = precision_score(test_Y[:, i], y_pred)
    recall = recall_score(test_Y[:, i], y_pred)
    
    print(token_order[i])
    print(
        precision,
        recall
    )
    print()
    

(88416, 4)
fast_trill_6khz
0.19254658385093168 0.29523809523809524

nr_syllable_3khz
0.6472222222222223 0.6246648793565683

triangle_3khz
0.25862068965517243 0.11811023622047244

upsweep_500hz
0.2641509433962264 0.16279069767441862



### AL loop

In [11]:
def least_confidence_sampling(y_pred, unlabelled_X, pool_size):
    # Calculate least confidence (1 - max probability)
    least_confidence_scores = 1 - np.max(y_pred, axis=1)

    # Select the least confident instances
    uncertain_indices = np.argsort(least_confidence_scores)[:pool_size]
    return uncertain_indices

def precision_recall(y_true, y_pred):
    classwize = {}
    for i in range(4):
        m = keras.metrics.Precision(thresholds=0.5)
        m.update_state(y_true[:, i], y_pred[:, i])
        precision = m.result().numpy()
        
        m = keras.metrics.Recall(thresholds=0.5)
        m.update_state(y_true[:, i], y_pred[:, i])
        recall = m.result().numpy()
        
        classwize[i] = (precision, recall)
    
    return classwize    

model, iteration_metrics, histories = AL_simulation(
    
        unlabelled=(unlabelled_X, unlabelled_Y), 
        intial_labelled=(labelled_X, labelled_Y), 
        test=(test_X, test_Y),
        model=model, 
        name='least_confidence_sampling', 
        evaluate_metrics=precision_recall,
        pool_size= int(len(unlabelled_X) / 20), # iterations
        num_iterations=20,
        sampling_query=least_confidence_sampling
        
)

 --- Iteration 1 --- 
--- Epoch 1 ---
previous shapes:  (70733, 1024) (70733, 4)


new shapes:  (36015, 1024) (36015, 4)

Epoch 1: saving model to /home/ec2-user/acoustic-AL/models/least_confidence_sampling/training/checkpoint.weights.h5
1126/1126 - 4s - loss: 0.0320 - recall_2: 0.0460 - precision_2: 0.4000 - auc_pr: 0.1245 - metric/map: 0.0225 - 4s/epoch - 3ms/step
--- Epoch 2 ---
previous shapes:  (70733, 1024) (70733, 4)
new shapes:  (36015, 1024) (36015, 4)

Epoch 1: saving model to /home/ec2-user/acoustic-AL/models/least_confidence_sampling/training/checkpoint.weights.h5
1126/1126 - 3s - loss: 0.0256 - recall_2: 0.1030 - precision_2: 0.7181 - auc_pr: 0.2901 - metric/map: 0.0247 - 3s/epoch - 3ms/step
--- Epoch 3 ---
previous shapes:  (70733, 1024) (70733, 4)
new shapes:  (36015, 1024) (36015, 4)

Epoch 1: saving model to /home/ec2-user/acoustic-AL/models/least_confidence_sampling/training/checkpoint.weights.h5
1126/1126 - 3s - loss: 0.0243 - recall_2: 0.1462 - precision_2: 0.7170 - auc_pr: 0.3344 - metric/map: 0.0250 - 3s/epoch - 3ms/step
--- Epoch 4 ---
previous

2024-09-23 05:12:56.626090: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1158885376 exceeds 10% of free system memory.


5456/8842 [=================>............] - ETA: 3s

KeyboardInterrupt: 

In [24]:
pred_Y = model(test_X)

# look at all precisions and recalls
for i in range(4):
    m = keras.metrics.Precision(thresholds=0.5)
    m.update_state(test_Y[:, i], pred_Y[:, i])
    print(m.result())

    m = keras.metrics.Recall(thresholds=0.5)
    m.update_state(test_Y[:, i], pred_Y[:, i])
    print(m.result())
    print("")

tf.Tensor(0.4834437, shape=(), dtype=float32)
tf.Tensor(0.7053292, shape=(), dtype=float32)
tf.Tensor(0.30769232, shape=(), dtype=float32)
tf.Tensor(0.3271028, shape=(), dtype=float32)


In [ ]:
def get_predictions(model, batched_test_dataset):
    y_true, y_pred = [], []
    for batch_X, batch_y in batched_test_dataset, desc='loading/predicting test ds':
        predictions = model(batch_X, training=False)
        y_true.extend(batch_y.numpy())
        y_pred.extend(predictions.numpy()) 

    return np.array(y_true), np.array(y_pred)


def evaluate_metrics(y_true, y_pred):
    y_true_flat = y_true.reshape(-1, 4)
    y_pred_flat = y_pred.reshape(-1, 4)
    
    m = keras.metrics.AUC(curve='roc')
    m.update_state(y_true_flat[:, 0], y_pred_flat[:, 0])
    m.result()
    
    m = keras.metrics.AUC(curve='pr')
    m.update_state(y_true_flat[:, 0], y_pred_flat[:, 0])
    m.result()
    
    for i in range(4):
        print('\n class ', i)
        y_pred_binary = (y_pred_flat[:, i] >= threshold).astype(int)

        precision_metric = Precision()
        recall_metric = Recall()

        precision_metric.update_state(y_true_flat[:, i], y_pred_binary)
        recall_metric.update_state(y_true_flat[:, i], y_pred_binary)

        precision = precision_metric.result().numpy()
        recall = recall_metric.result().numpy()

        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        
        ...
            